In [36]:
import bentoml
import pandas as pd
import os
import numpy as np



In [37]:
dataset = pd.read_csv(os.path.abspath(os.path.join(os.getcwd() + "\\..\\..\\data\\Housing.csv")))

# Printing first 5 records of the dataset
print(dataset.head(5))


      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  


In [38]:
from sklearn.preprocessing import OneHotEncoder

s = (dataset.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)
print('No. of. categorical features: ', 
      len(object_cols))

Categorical variables:
['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']
No. of. categorical features:  7


In [39]:
OH_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
OH_cols = pd.DataFrame(OH_encoder.fit_transform(dataset[object_cols]))
OH_cols.index = dataset.index
OH_cols.columns = OH_encoder.get_feature_names_out()
df_final = dataset.drop(object_cols, axis=1)
df_final = pd.concat([df_final, OH_cols], axis=1)

In [40]:
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

X = df_final.drop(['price'], axis=1)
Y = df_final['price']

# Split the training set into 
# training and validation set
X_train, X_valid, Y_train, Y_valid = train_test_split(
    X, Y, train_size=0.8, test_size=0.2, random_state=0)

In [41]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_percentage_error

model_SVR = svm.SVR()
model_SVR.fit(X_train,Y_train)
Y_pred = model_SVR.predict(X_valid)

print(mean_absolute_percentage_error(Y_valid, Y_pred))

0.2710074432862681


In [42]:
from sklearn.linear_model import LinearRegression

model_LR = LinearRegression()
model_LR.fit(X_train, Y_train)
Y_pred = model_LR.predict(X_valid)

print(mean_absolute_percentage_error(Y_valid, Y_pred))

0.1603519515522061


In [43]:
saved_model  = bentoml.sklearn.save_model("bento_gs_model", model_LR)
saved_model

Model(tag="bento_gs_model:q2hjhefwfgq2ehgo", path="C:\Users\EdwinVivekN\bentoml\models\bento_gs_model\q2hjhefwfgq2ehgo\")

In [44]:
from sklearn.base import BaseEstimator
model: BaseEstimator = bentoml.sklearn.load_model("bento_gs_model:latest")
model

LinearRegression()

In [45]:
!bentoml models list

 Tag                            Module           Size      Creation Time       
 bento_gs_model:q2hjhefwfgq2eâ€¦  bentoml.sklearn  1.90 KiB  2024-12-09 18:01:33 
 bento_gs_model:3pjpadnvog2psâ€¦  bentoml.sklearn  1.90 KiB  2024-12-08 20:06:49 


In [46]:
!bentoml models get "bento_gs_model"
#!bentoml models delete "bento_gs_model"

name: bento_gs_model
version: q2hjhefwfgq2ehgo
module: bentoml.sklearn
labels: {}
options: {}
metadata: {}
context:
  framework_name: sklearn
  framework_versions:
    scikit-learn: 1.5.2
  bentoml_version: 1.3.15
  python_version: 3.12.8
signatures:
  predict:
    batchable: false
api_version: v1
creation_time: '2024-12-09T12:31:33.922998+00:00'



In [47]:
bento_model = bentoml.models.get("bento_gs_model:latest")
bento_model

Model(tag="bento_gs_model:q2hjhefwfgq2ehgo", path="C:\Users\EdwinVivekN\bentoml\models\bento_gs_model\q2hjhefwfgq2ehgo")

In [48]:
my_runner = bento_model.to_runner()
my_runner

C:\Users\EdwinVivekN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\bentoml\_internal\models\model.py:367: BentoMLDeprecationWarning: `get_runnable` is deprecated since BentoML v1.4 and will be removed in a future version. Use `get_service` instead.
  self._runnable = self.info.imported_module.get_runnable(self)
C:\Users\EdwinVivekN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\bentoml\_internal\models\model.py:354: BentoMLDeprecationWarning: `Runner` is deprecated since BentoML v1.4 and will be removed in a future version. Please upgrade to new style services.
  return Runner(


Runner(name='bento_gs_model', models=[Model(tag="bento_gs_model:q2hjhefwfgq2ehgo", path="C:\Users\EdwinVivekN\bentoml\models\bento_gs_model\q2hjhefwfgq2ehgo")], resource_config=None, runnable_class=<class '_bentoml_impl.frameworks.sklearn.get_runnable.<locals>.SklearnRunnable'>, embedded=False, runner_methods=[RunnerMethod(runner=..., name='predict', config=RunnableMethodConfig(batchable=False, batch_dim=(0, 0), input_spec=None, output_spec=None, is_stream=False), max_batch_size=100, max_latency_ms=60000, doc=None)], scheduling_strategy=<class 'bentoml._internal.runner.strategy.DefaultStrategy'>, workers_per_resource=1, runnable_init_params={}, _runner_handle=<bentoml._internal.runner.runner_handle.DummyRunnerHandle object at 0x000001D10177A2D0>)

In [49]:
# Initialize the Runner in the current process (for development and testing only):
my_runner.init_local()

'Runner.init_local' is for debugging and testing only. Make sure to remove it before deploying to production.


In [50]:
# Use the runner for inference 
predictions = my_runner.predict.run(X_valid)
predictions

array([ 3950288.61876183,  6173868.81883112,  4483635.98836252,
        7258732.75105289,  2836727.58490482,  7032947.09749074,
        3203851.47112392,  3270994.00904054,  3472554.03645915,
        8289978.32623725,  6605321.62954623,  3723366.23684093,
        3812376.95976084,  4548966.84544609,  4020476.34849668,
        1969836.22090135,  4057262.98087852,  3704586.86711741,
        3282767.93188791,  4609423.64909564,  5968243.73637161,
        6363698.62063815,  4751300.32389   ,  2659595.27633028,
        5305573.24662116,  5680819.5878446 ,  5404106.90027142,
        5543050.52192535,  5768360.47982208,  5801753.70839283,
        3389277.96110609,  6399092.02678423,  7081030.31411775,
        2913042.40387669,  4498664.01335422,  5210561.68059368,
        5013457.8412231 ,  3707596.71347574,  2916603.4548534 ,
        3937761.75634065,  8041334.20180917,  4942174.61142059,
        6432605.2198175 ,  3511338.78156424,  3813475.39540795,
        6434856.19540026,  4447687.02885

In [105]:
#to load mlflow model
bento_model2 = bentoml.mlflow.load_model("bento_gs_model:latest")
bento_model2
pred = bento_model2.predict(X_valid[:1])
pred

NotFound: Model bento_gs_model:q2hjhefwfgq2ehgo was saved with module bentoml.sklearn, not loading with bentoml.mlflow.

In [51]:
svc = bentoml.Service("bento_gs_model", runners=[my_runner])

C:\Users\EdwinVivekN\AppData\Local\Temp\ipykernel_6376\4227574256.py:1: BentoMLDeprecationWarning: `bentoml.Service` is deprecated since BentoML v1.4 and will be removed in a future version. Please upgrade to @bentoml.service().
  svc = bentoml.Service("bento_gs_model", runners=[my_runner])


In [81]:
input_d = [4000, 3, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0]
my_runner.predict.run(np.asarray(input_d).reshape(1,-1))

C:\Users\EdwinVivekN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([3950288.61876183])

In [ ]:
!bentoml serve service:MyModel

In [102]:
!bentoml build 

INFO: Adding current BentoML version to requirements.txt: bentoml==1.3.15
INFO: Locking PyPI package versions.
INFO: Locking packages for x86_64-unknown-linux-gnu. Pass `--platform` option to specify the platform.

â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ•— â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ•—â–ˆâ–ˆâ–ˆâ•—   â–ˆâ–ˆâ•—â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ•— â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ•— â–ˆâ–ˆâ–ˆâ•—   â–ˆâ–ˆâ–ˆâ•—â–ˆâ–ˆâ•—
â–ˆâ–ˆâ•”â•�â•�â–ˆâ–ˆâ•—â–ˆâ–ˆâ•”â•�â•�â•�â•�â•�â–ˆâ–ˆâ–ˆâ–ˆâ•—  â–ˆâ–ˆâ•‘â•šâ•�â•�â–ˆâ–ˆâ•”â•�â•�â•�â–ˆâ–ˆâ•”â•�â•�â•�â–ˆâ–ˆâ•—â–ˆâ–ˆâ–ˆâ–ˆâ•— â–ˆâ–ˆâ–ˆâ–ˆâ•‘â–ˆâ–ˆâ•‘
â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ•”â•�â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ•—  â–ˆâ–ˆâ•”â–ˆâ–ˆâ•— â–ˆâ–ˆâ•‘   â–ˆâ–ˆâ•‘   â–ˆâ–ˆâ•‘   â–ˆâ–ˆâ•‘â–ˆâ–ˆâ•”â–ˆâ–ˆâ–ˆâ–ˆâ•”â–ˆâ–ˆâ•‘â–ˆâ–ˆâ•‘
â–ˆâ–ˆâ•”â•�â•�â–ˆâ–ˆâ•—â–ˆâ–ˆâ•”â•�â•�â•�  â–ˆâ–ˆâ•‘â•šâ–ˆâ–ˆâ•—â–ˆâ–ˆâ•‘   â–ˆâ–ˆâ•‘   â–ˆâ–ˆâ•‘   â–ˆâ–ˆâ•‘â–ˆâ–ˆâ•‘â•šâ–ˆâ–ˆâ•”â•�â–ˆâ–ˆâ•‘â–ˆâ–ˆâ•‘
â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ•”â•�â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ•—â–ˆâ–ˆâ•‘ â•šâ–ˆâ–ˆâ–ˆâ–ˆâ•‘   â–ˆâ–ˆâ•‘   â•šâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ•”â•�â–ˆâ–ˆâ•‘ â•šâ•�â•� â–ˆâ–ˆâ•‘â–ˆâ–ˆâ–

In [103]:
!bentoml list

 Tag                        Size       Model Size  Creation Time       
 my_model:umv2rsnwgg2hahgo  18.07 KiB  1.90 KiB    2024-12-09 18:59:39 
 my_model:acngxbnwgg2lchgo  18.07 KiB  1.90 KiB    2024-12-09 18:55:12 


In [104]:
!bentoml get "my_model"
#!bentoml delete

service: service:MyModel
name: my_model
version: umv2rsnwgg2hahgo
bentoml_version: 1.3.15
creation_time: '2024-12-09T13:29:39.033304+00:00'
labels:
  owner: ed
  stage: dev
models: []
entry_service: MyModel
services:
- name: MyModel
  service: ''
  models:
  - tag: bento_gs_model:q2hjhefwfgq2ehgo
    module: bentoml.sklearn
    creation_time: '2024-12-09T12:31:33.922998+00:00'
  dependencies: []
  config:
    resources:
      cpu: '2'
    traffic:
      timeout: 10
envs: []
schema:
  name: MyModel
  type: service
  routes:
  - name: predict
    route: /predict
    batchable: false
    input:
      properties:
        input_data:
          format: numpy-array
          title: Input Data
          type: tensor
      required:
      - input_data
      title: Input
      type: object
    output:
      format: numpy-array
      type: tensor
    is_task: false
spec: 1
runners: []
apis: []
docker:
  distro: debian
  python_version: '3.12'
  cuda_version: null
  env: null
  system_packages: nu